In [1]:
GROQ_API_KEY="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"

## 1. Smart Highlights

In [2]:
from groq import Groq

client = Groq(api_key=GROQ_API_KEY)
def get_groq_mistral_response(prompt: str) -> str:
    try:
        response = client.chat.completions.create(
            model="mistral-saba-24b",
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            temperature=1,
            max_completion_tokens=1024,
            top_p=1,
            stream=False,
            stop=None,    
        )
    except Exception as e:
        print(f"Groq Error: {e}")
        return "⚠️ Error from Groq/Mistral API."

    return response.choices[0].message.content

In [3]:
def smart_highlight_prompt(text: str) -> str:
    return f"""
    You are an advanced document annotator with expertise in analyzing and classifying highlighted text. Your task is to provide a **precise and insightful annotation** for the given highlight.

    1. **Classify the Highlight Type**:  
       Choose from the following categories:
       - **Concept** (A single idea, definition, or principle)  
       - **Process** (Step-by-step explanation or structured reasoning)  
       - **Fact** (A verified statement, research finding, or statistic)  
       - **Task** (A to-do item or action-based instruction)  
       - **Formula** (A structured or mathematical representation)  

    2. **Classify the Sentence Type** (Choose the best match):  
       - **Definition**  
       - **Explanation**  
       - **Quote**  
       - **Important Fact**  
       - **Recommendation**  
       - **Author's Opinion**  
       - **Analogy**  
       - **Other (Specify if needed)**  

    3. **Generate a Short Note**:  
       - Clearly **explain the significance** of the highlight in 1-2 sentences.  
       - Avoid generic phrases like "This sentence describes..."  
       - Use **engaging and direct** phrasing: "It highlights...", "It defines...", "It recommends...".  

    **Highlighted Sentence:**  
    \"\"\"{text}\"\"\"

    **Respond ONLY in this format (no extra text):**  
    - **Highlight Type**: [One of Concept, Process, Fact, Task, Formula]  
    - **Sentence Type**: [One of Definition, Explanation, Quote, etc.]  
    - **Short Note**: [A brief, professional annotation]  
    """


# def smart_highlight_prompt(text: str) -> str:
#    return f"""
#    You are an advanced document annotator with expertise in analyzing and classifying highlighted sentences. Your task is to provide a professional and insightful annotation for the given highlighted sentence. Follow these steps:

#    1. **Classify the Highlight**: Determine whether the highlighted text is a "Concept" (a concise idea or definition) or a "Paragraph" (a more detailed explanation or narrative). 
#       - If the highlighted text is a "Concept" (e.g., "Deep Learning"), provide a short and precise definition or explanation.
#       - If the highlighted text is a "Paragraph", generate a concise TL;DR summary that captures the essence of the content.

#    2. **Classify the Sentence**: Determine the type of the highlighted sentence from the following categories:
#       - Summary
#       - Definition
#       - Explanation
#       - Quote
#       - Important Fact
#       - Recommendation
#       - Author's Opinion
#       - Analogy
#       - Other (specify if none of the above)

#    3. **Provide a Short Note**: Write a concise and user-friendly annotation that explains the essence of the highlighted sentence. Avoid generic phrases like "This sentence describes...". Instead, use direct and meaningful language such as "It highlights...", "It defines...", or "It recommends...".

#    4. **Ensure Clarity and Precision**: Your response should be clear, precise, and helpful to the user. Avoid overly complex language or unnecessary elaboration.

#    Highlighted Sentence:
#    \"\"\"{text}\"\"\"

#    Respond with:
#    - **Highlight Type**: (Concept or Paragraph)
#    - **Sentence Type**: (e.g., Explanation, Quote)
#    - **Short Note**: (A one-sentence annotation that captures the essence of the highlighted sentence in a professional and insightful manner)

#    Make your response polished, accurate, and user-friendly.
#    """


In [5]:
# Concept
highlight1 = "Reinforcement learning uses reward feedback to teach agents how to act."
prompt = smart_highlight_prompt(highlight1)
print(get_groq_mistral_response(prompt))

- **Highlight Type**: Concept
- **Sentence Type**: Definition
- **Short Note**: It defines reinforcement learning as a method that uses reward feedback to train agents on performing tasks.


In [6]:
# For paragraph
highlight2 = "Float16 can only represent numbers up to 65504, whilst bfloat16 can represent huge numbers up to 10^38! But notice both number formats use only 16bits! This is because float16 allocates more bits so it can represent smaller decimals better, whilst bfloat16 cannot represent fractions well. But why float16? Let's just use float32! But unfortunately float32 in GPUs is very slow for matrix multiplications - sometimes 4 to 10x slower! So we cannot do this."

prompt = smart_highlight_prompt(highlight2)
print(get_groq_mistral_response(prompt))

- **Highlight Type**: Concept
- **Sentence Type**: Explanation
- **Short Note**: It highlights the trade-offs and efficiency considerations between float16 and bfloat16 compared to float32 for representing numerical values, especially in the context of GPU performance during matrix multiplications.


## 2. Semantic summary prompt

In [ ]:
def semantic_summary_prompt(highlights: list) -> str:
  return f"""
    You are an AI-powered summarization expert. Your task is to analyze the provided highlights and generate a **concise study memory** in a well-structured, high-impact paragraph.

    ### **Instructions:**
    1. **Capture the key insights** from the highlights without repeating information.  
    2. **Group related ideas** for a logical flow (e.g., learning techniques, AI advancements, safety concerns).  
    3. **Make it clear, precise, and engaging**, avoiding unnecessary filler words.  
    4. **Ensure natural readability** – the summary should feel polished and professional.

    ### **Highlights to Summarize:**  
    {highlights}

    ### **Respond in the following format:**  
    - **Compressed Study Memory:**  
      - [A single, structured paragraph summarizing the highlights effectively]
    """

In [ ]:
highlight1 = "Backpropagation is a supervised learning algorithm used to train neural networks by adjusting weights based on error feedback."
highlight2 = "The activation function in a neural network determines whether a neuron should be activated based on weighted inputs."
highlight3 = "Reinforcement learning allows an agent to learn by receiving rewards or penalties, optimizing future decisions based on past actions." 
highlight4 = "Gradient descent is an optimization algorithm that adjusts model parameters by computing the gradient of the loss function."
highlight5 = "As Geoffrey Hinton once said, 'Deep learning is the first technology in history that can solve perceptual problems better than humans.'"
highlight6 = "Elon Musk argues that AI safety is humanity’s biggest challenge, requiring careful regulation to prevent catastrophic outcomes." 
highlight7 = "Neural networks with more than three hidden layers are commonly referred to as deep neural networks."
highlight8 = "In 2023, OpenAI's ChatGPT reached 100 million users within just two months, making it the fastest-growing app in history."
highlight9 = "To improve model generalization, always use dropout regularization to prevent overfitting in deep learning models."
highlight10 = "For high-performance AI models, use a combination of transfer learning and fine-tuning on domain-specific datasets."

highlights = [highlight1, highlight2, highlight3, highlight4, highlight5, highlight6, highlight7, highlight8, highlight9, highlight10]
prompt = semantic_summary_prompt(highlights)
print(get_groq_mistral_response(prompt))

- **Compressed Study Memory:**
  Machine learning encompasses various techniques including backpropagation, which trains neural networks through error feedback and weight adjustments, and reinforcement learning, where models learn by receiving rewards or penalties. Activation functions determine neuron activation in neural networks, while gradient descent optimizes model parameters. Deep learning, surpassing human capabilities in perceptual tasks, involves deep neural networks with multiple hidden layers. Ensuring model generalizability requires dropout regularization to prevent overfitting, and combining transfer learning with fine-tuning on domain-specific datasets enhances performance. Notable milestones include OpenAI's ChatGPT hitting 100 million users in two months, highlighting AI's rapid growth. Safety remains a pressing concern; Elon Musk emphasizes stringent regulation to mitigate potential catastrophic risks.


## 3. ML based Auto-tagging

In [11]:
import spacy

# Load a more powerful transformer-based model
nlp = spacy.load("en_core_web_trf")  # More accurate than "en_core_web_sm"

# Sample notes
notes = [
    "Backpropagation is used in neural networks.",
    "Here’s a cool formula I found: E = mc².",
    "Important deadline: Submit research paper by Friday.",
    "In 2023, OpenAI’s ChatGPT became the fastest-growing app in history.",
    "The sigmoid activation function is used in logistic regression.",
    "Elon Musk warns about the risks of AI surpassing human intelligence."
]

def extract_entities(text):
    """Extract named entities using a transformer-based model."""
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

# Process each note
for note in notes:
    print(f"\n🔹 **Note:** {note}")
    print("📌 **Detected Entities:**", extract_entities(note))


/home/vivek/miniforge3/envs/NCvenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



🔹 **Note:** Backpropagation is used in neural networks.
📌 **Detected Entities:** []

🔹 **Note:** Here’s a cool formula I found: E = mc².
📌 **Detected Entities:** []

🔹 **Note:** Important deadline: Submit research paper by Friday.
📌 **Detected Entities:** [('Friday', 'DATE')]

🔹 **Note:** In 2023, OpenAI’s ChatGPT became the fastest-growing app in history.
📌 **Detected Entities:** [('2023', 'DATE'), ('OpenAI', 'ORG'), ('ChatGPT', 'PRODUCT')]

🔹 **Note:** The sigmoid activation function is used in logistic regression.
📌 **Detected Entities:** []

🔹 **Note:** Elon Musk warns about the risks of AI surpassing human intelligence.
📌 **Detected Entities:** [('Elon Musk', 'PERSON')]


In [1]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap.umap_ import UMAP
from hdbscan import HDBSCAN

# Preprocess the text (e.g., lowercasing, removing stopwords, etc.)
def preprocess_text(docs):
    return [doc.lower() for doc in docs]

docs = [
    "Backpropagation is used in neural networks.",
    "Submit the report by Friday.",
    "Einstein's equation: E=mc².",
    "Meeting with Dr. Smith at 3 PM.",
    "Neural networks can learn complex patterns.",
    "Activation functions like ReLU and Sigmoid are crucial.",
    "Machine learning models improve with more data.",
    "Quantum physics is fascinating and complex.",
    "Newton formulated the laws of motion.",
    "The deadline for the project is next Monday.",
    "Discuss the new AI model with the team.",
    "A research paper on AI ethics was published.",
    "Data preprocessing is a key step in ML pipelines.",
    "Professor John teaches deep learning at MIT.",
    "Python is widely used for data science.",
    "E=mc² describes the relationship between energy and mass.",
    "Meeting scheduled at 2 PM with the research team."
]


# Preprocess the documents
processed_docs = preprocess_text(docs)

# Use a custom embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Fine-tune UMAP settings
umap_model = UMAP(n_neighbors=15, n_components=5, metric="cosine", random_state=42)

# Fine-tune HDBSCAN settings
hdbscan_model = HDBSCAN(min_cluster_size=2, min_samples=1, metric="euclidean", cluster_selection_method="eom")

# Initialize BERTopic with custom models
topic_model = BERTopic(embedding_model=embedding_model, umap_model=umap_model, hdbscan_model=hdbscan_model)

# Fit and transform the documents
topics, probs = topic_model.fit_transform(processed_docs)

# Print the topics and their keywords
topic_model.get_topic_info()


/home/vivek/miniforge3/envs/NCvenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Topic,Count,Name,Representation,Representative_Docs
0,0,5,0_complex_networks_neural_and,"[complex, networks, neural, and, is, are, deep...","[quantum physics is fascinating and complex., ..."
1,1,5,1_data_ai_with_is,"[data, ai, with, is, improve, discuss, model, ...","[python is widely used for data science., data..."
2,2,4,2_the_meeting_pm_at,"[the, meeting, pm, at, with, dr, friday, proje...","[the deadline for the project is next monday.,..."
3,3,3,3_mc²_formulated_motion_mass,"[mc², formulated, motion, mass, of, relationsh...","[einstein's equation: e=mc²., e=mc² describes ..."


In [2]:
for i, doc in enumerate(docs):
    topic_name = topic_model.get_topic(topics[i])
    topic_keywords = ", ".join([word for word, _ in topic_name]) if topic_name else "No Topic"
    print(f"📌 '{doc}' → Tag: #{topic_keywords}")


📌 'Backpropagation is used in neural networks.' → Tag: #complex, networks, neural, and, is, are, deep, fascinating, functions, crucial
📌 'Submit the report by Friday.' → Tag: #the, meeting, pm, at, with, dr, friday, project, monday, next
📌 'Einstein's equation: E=mc².' → Tag: #mc², formulated, motion, mass, of, relationship, einstein, laws, describes, equation
📌 'Meeting with Dr. Smith at 3 PM.' → Tag: #the, meeting, pm, at, with, dr, friday, project, monday, next
📌 'Neural networks can learn complex patterns.' → Tag: #complex, networks, neural, and, is, are, deep, fascinating, functions, crucial
📌 'Activation functions like ReLU and Sigmoid are crucial.' → Tag: #complex, networks, neural, and, is, are, deep, fascinating, functions, crucial
📌 'Machine learning models improve with more data.' → Tag: #data, ai, with, is, improve, discuss, model, key, ml, machine
📌 'Quantum physics is fascinating and complex.' → Tag: #complex, networks, neural, and, is, are, deep, fascinating, functions, 

In [7]:
import spacy

# Load spaCy's pre-trained NER model
nlp = spacy.load("en_core_web_trf")

# Sample note
text = "Meeting with Elon Musk at SpaceX HQ in Hawthorne on 2024-03-15. Discussed Mars colonization timelines."

# Process the text
doc = nlp(text)

# Extract entities
tags = {ent.label_: [] for ent in doc.ents}
for ent in doc.ents:
    tags[ent.label_].append(ent.text)

print("Auto-generated tags:", tags)

Auto-generated tags: {'PERSON': ['Elon Musk'], 'ORG': ['SpaceX'], 'GPE': ['Hawthorne'], 'DATE': ['2024-03-15'], 'LOC': ['Mars']}


In [15]:
# Auto-generated tags: {'PERSON': ['Elon Musk'], 'ORG': ['SpaceX'], 'GPE': ['Hawthorne'], 'DATE': ['2024-03-15'], 'LOC': ['Mars']}
text = "Backpropagation is used in neural networks."
nlp = spacy.load("en_core_web_trf")
doc = nlp(text)
tags = {ent.label_: [] for ent in doc.ents}
for ent in doc.ents:
    tags[ent.label_].append(ent.text)

print("Auto-generated tags:", tags)

Auto-generated tags: {}


In [3]:
import spacy
from rake_nltk import Rake
from sentence_transformers import SentenceTransformer, util

# Initialize tools
nlp = spacy.load("en_core_web_trf")
rake = Rake()
model = SentenceTransformer('all-MiniLM-L6-v2')

# Input sentence
sentence = "Google is developing a new quantum computing chip in collaboration with NASA."

# ---------- Step 1: NER ----------
doc = nlp(sentence)
ner_tags = list(set(ent.text for ent in doc.ents))

# ---------- Step 2: Keyword Extraction ----------
rake.extract_keywords_from_text(sentence)
rake_tags = rake.get_ranked_phrases()

# ---------- Step 3: Semantic Tag Matching ----------
# Predefined tag candidates (can be domain-specific or broad)
candidate_tags = [
    "Quantum Computing", "AI", "Google", "NASA", "Technology", "Hardware",
    "Space", "Machine Learning", "Semiconductor", "Research", "Partnership"
]

# Encode sentence and candidate tags
sentence_embedding = model.encode(sentence, convert_to_tensor=True)
tag_embeddings = model.encode(candidate_tags, convert_to_tensor=True)

# Compute cosine similarities
cosine_scores = util.cos_sim(sentence_embedding, tag_embeddings)[0]

# Threshold filtering (tweak as needed)
semantic_tags = [
    candidate_tags[i]
    for i in range(len(candidate_tags))
    if cosine_scores[i] > 0.45  # threshold for semantic match
]

# ---------- Combine All ----------
# all_tags = list(set(ner_tags + rake_tags + semantic_tags))

# print("📌 Final Auto-Generated Tags:")
# for tag in all_tags:
#     print("-", tag)

# Combine all tags and normalize
raw_tags = ner_tags + rake_tags + semantic_tags

# Clean + filter
filtered_tags = []
seen = set()
for tag in raw_tags:
    tag_clean = tag.strip().lower()
    if tag_clean not in seen and len(tag_clean) > 2 and tag_clean not in {"developing", "collaboration", "new"}:
        seen.add(tag_clean)
        filtered_tags.append(tag_clean.title())  # Title case for readability

# Output
print("📌 Final Cleaned Auto-Generated Tags:")
for tag in filtered_tags:
    print("-", tag)


📌 Final Cleaned Auto-Generated Tags:
- Google
- Nasa
- New Quantum Computing Chip
- Quantum Computing


## 4. Cross-Referencing(Multi-doc magic)

In [1]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions
import uuid

# ✅ Initialize embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# ✅ Initialize ChromaDB with persistent storage
chroma_client = chromadb.PersistentClient(path="data/chroma_db")

# ✅ Get or create the 'notes' collection
collection = chroma_client.get_or_create_collection(name="notes")

/home/vivek/miniforge3/envs/NCvenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ✅ Add new note function
def add_note(text, metadata=None):
    note_id = str(uuid.uuid4())  # Unique ID
    embedding = embedding_model.encode(text)
    collection.add(
        documents=[text],
        embeddings=[embedding],
        ids=[note_id],
        metadatas=[metadata or {}]
    )
    return note_id, embedding

In [3]:
# ✅ Similar notes search
def find_similar_notes(query_embedding, current_note_id=None, top_k=3):
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k + 1  # Fetch extra to allow filtering
    )
    
    # Filter out current note
    filtered_notes = []
    for note_id, doc in zip(results["ids"][0], results["documents"][0]):
        if note_id != current_note_id:
            filtered_notes.append(doc)
        if len(filtered_notes) >= top_k:
            break

    return filtered_notes


# ✅ Example usage
new_note = "GANs generate realistic images."
note_id, embedding = add_note(new_note, metadata={"type": "note", "topic": "AI"})


In [4]:
new_note = "VAEs also generate images, but their latent space is continuous."
note_id, embedding = add_note(new_note, metadata={"type": "note", "topic": "AI"})

In [ ]:
add_note("Transformers revolutionized NLP tasks.", metadata={"type": "note", "topic": "AI"}) 
add_note("GANs are useful for data augmentation.", metadata={"type": "note", "topic": "AI"})

In [8]:
similar = find_similar_notes(embedding, current_note_id=note_id)

print("🔍 Similar Notes:")
for i, note in enumerate(similar):
    print(f"{i+1}. {note}")


🔍 Similar Notes:
1. GANs generate realistic images.
2. GANs are useful for data augmentation.
3. Transformers revolutionized NLP tasks.
